# Test the deployed web application

In this notebook, we test the local web app running inside the Docker container we made previously.

In [1]:
import os
import numpy as np
import requests
import pandas as pd
import json
from timer import elapsed

from utilities import text_to_json
%load_ext dotenv
%dotenv

/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
image_name = os.getenv('docker_login') + os.getenv('image_repo')
image_name

'kwarodom/mlaksdep'

Run the Docker conatainer in the background and open port 80.

In [10]:
%%bash --bg -s "$image_name"
docker run -p 5000:5000 $1

Wait a few seconds for the application to spin up and then check that everything works.

In [12]:
!curl "http://0.0.0.0:5000/"

curl: (7) Failed to connect to 0.0.0.0 port 5000: Connection refused


In [5]:
!ping 0.0.0.0

PING 0.0.0.0 (127.0.0.1) 56(84) bytes of data.
64 bytes from 127.0.0.1: icmp_seq=1 ttl=64 time=0.026 ms
64 bytes from 127.0.0.1: icmp_seq=2 ttl=64 time=0.039 ms
64 bytes from 127.0.0.1: icmp_seq=3 ttl=64 time=0.037 ms
64 bytes from 127.0.0.1: icmp_seq=4 ttl=64 time=0.039 ms
^C

--- 0.0.0.0 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3056ms
rtt min/avg/max/mdev = 0.026/0.035/0.039/0.006 ms


In [6]:
!curl "http://0.0.0.0:80/version"

curl: (7) Failed to connect to 0.0.0.0 port 80: Connection refused


Now, let's use one of the duplicate questions to test our driver.

In [7]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"getting the length of a 'named' array?. i'm not sure what they are called, but what i mean is this:  length should be 2 here how can i see how many attributes i have in the array? array.length doesn't work =( i've been trying all kinds of things and i feel like i'm missing something really simple here.. thank you for your help"

In [8]:
jsontext = text_to_json(text_to_score)
jsontext[:100]

'{"input": "getting the length of a \'named\' array?. i\'m not sure what they are called, but what i mea'

In [9]:
headers = {'content-type': 'application/json'}
%time r = requests.post('http://0.0.0.0:80/score', data=jsontext, headers=headers)
print(r)
r.json()

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=80): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdc84629048>: Failed to establish a new connection: [Errno 111] Connection refused',))

NameError: name 'r' is not defined

Let's try a few more duplicate questions and display their top 3 original matches.

In [ ]:
dupes_to_score = dupes_test.iloc[:5,4]

In [ ]:
url = 'http://0.0.0.0:80/score'
results = [requests.post(url, data=text_to_json(text), headers=headers) for text in dupes_to_score]

Let's print top 3 matches for each duplicate question.

In [ ]:
[results[i].json()['result'][0][0:3] for i in range(0, len(results))]

Next let's quickly check what the request response performance is for the locally running Docker container.

In [ ]:
text_data = list(map(text_to_json, dupes_to_score)) # Retrieve the text data

In [ ]:
timer_results = [elapsed(requests.post, return_elapsed=True)(url, data=data, headers=headers)
                 for data in text_data]

In [ ]:
timer_results

In [ ]:
print('Average time taken: {0:4.2f} ms'.format(10**3 * np.mean(timer_results)))

In [13]:
%%bash
docker stop $(docker ps -q)

"docker stop" requires at least 1 argument.
See 'docker stop --help'.

Usage:  docker stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers


CalledProcessError: Command 'b'docker stop $(docker ps -q)\n'' returned non-zero exit status 1

We can now [deploy our web application](05_Deploy_On_AKS.ipynb) on AKS.